##  **Faze** - face & gaze detection for attention prediction and emotion tracking
#### Shifeng, May 2020
Run the code cells below sequentially. Importing packages might take 10 seconds or so.

In [ ]:
import time
import sys
cvpath = "/opt/ros/kinetic/lib/python2.7/dist-packages"
if cvpath in sys.path: sys.path.remove(cvpath)
import cv2
import numpy as np
import matplotlib.pyplot as plt

from util.visualize import plot_all, plot_emotions
from util.gaze import vector_to_pitchyaw, pitchyaw_to_vector
from util.emotions import get_emotion
from run_with_webcam import *

### Initialize your webcam, '0' is the default

In [ ]:
webcam = cv2.VideoCapture(0)

### The Detector
* Run the cell below to get started.
* Two cv2 windows will pop up.
    * *Might be hidden behind this program when launched*

In [ ]:
# pixel to cm conversion (image plane)
p = 7./1200
focal_len = 2.7

current_face, landmarks, right_eye, left_eye = None, None, None, None
smooth = 0.7

emotion_history = []

iteration = 0
while True:
    """
    Read from Webcam
    """
    _, frame_bgr = webcam.read()
    if frame_bgr is None:
        time.sleep(0.1)
        continue
    iteration += 1

    orig_frame = frame_bgr.copy()
    cv2.imshow("Webcam", orig_frame)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    frame_h,frame_w, _ = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    """
    Face detection + temporal smoothing of detection
    """
    # Haar cascade detector
    faces = face_cascade.detectMultiScale(gray)
    if len(faces):
        next_face = faces[0]
        if current_face is not None:
            current_face[:2] = smooth * next_face[:2] + (1 - smooth) * current_face[:2]
            current_face[3:] = smooth * next_face[3:] + (1 - smooth) * current_face[3:]
        else:
            current_face = next_face
    
    """
    Advanced Processing
    """
    emotion_frame = []
    if current_face is None:
        orig_frame = cv2.putText(orig_frame, 'No face detected', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    else:
        draw_cascade_face(current_face, orig_frame)
        
        """
        Emotion Detection
        """
        if iteration % 2 == 0:
            emotion = get_emotion(current_face, gray)
            # emotion smoothing
            if len(emotion_history) >= 3:
                emotion = emotion*0.5 + emotion_history[-1]*0.3 + emotion_history[-2]*0.2
            emotion_history.append(emotion)
            if len(emotion_history) > 61:
                del emotion_history[0]
            emotion_frame = plot_emotions(emotion_history)
                
            
        """
        Eye Landmark Detection
        """
        next_landmarks = detect_landmarks(current_face, gray)
        if landmarks is not None:
            landmarks = next_landmarks * smooth + (1 - smooth) * landmarks
        else:
            landmarks = next_landmarks
        draw_landmarks(landmarks, orig_frame)
        
        """
        Gaze Prediction
        """
        if landmarks is not None:
            eye_samples = segment_eyes(gray, landmarks)
            # eyenet inference
            eye_preds = run_eyenet(eye_samples)
            left_eyes = list(filter(lambda x: x.eye_sample.is_left, eye_preds))
            right_eyes = list(filter(lambda x: not x.eye_sample.is_left, eye_preds))

            if left_eyes:
                left_eye = smooth_eye_landmarks(left_eyes[0], left_eye, smoothing=0.2, gaze_smoothing=0.5)
            if right_eyes:
                right_eye = smooth_eye_landmarks(right_eyes[0], right_eye, smoothing=0.2, gaze_smoothing=0.5)
            
            gaze_vectors = np.empty((2,3))
            for ep in [left_eye, right_eye]:
                for (x, y) in ep.landmarks[16:33]:
                    color = (160, 230, 160)
#                     if ep.eye_sample.is_left:
#                         color = (230, 160, 160)
                    cv2.circle(orig_frame,(int(round(x)), int(round(y))), 1, color, -1, lineType=cv2.LINE_AA)

                gaze = ep.gaze.copy()
                if ep.eye_sample.is_left:
                    gaze[1] = -gaze[1]
                    gaze_vectors[0,:] = pitchyaw_to_vector(np.expand_dims(gaze,0))
                else:
                    gaze_vectors[1,:] = pitchyaw_to_vector(np.expand_dims(gaze,0))
                util.gaze.draw_gaze(orig_frame, ep.landmarks[-2], gaze, length=60.0, thickness=2)
    
    """
    Visualization
    """
    plt_frame = []
    if len(faces):
        (x,y,w,h) = current_face
        face_size = abs(w - h)/4 + min(w,h)
        face_distance = 5960/face_size    
        face_img_frame = np.array([(x-frame_w/2)*p, (y-frame_h/2)*p, 2.7])
        face_world_frame = face_img_frame/np.linalg.norm(face_img_frame) * face_distance
        plt_frame = plot_all(face_world_frame, gaze_vectors)
    
    if len(plt_frame):
        cv2.imshow("3D Plot", plt_frame)
        
    if len(emotion_frame):
        cv2.imshow("Emotion tracking", emotion_frame)
    
    cv2.imshow("Webcam", orig_frame)
    
    time.sleep(0.04)
    cv2.waitKey(1)

### Release camera and close all cv2 windows at the end

In [ ]:
webcam.release()
cv2.destroyAllWindows()

---
# Miscellaneous

## Webcam test

In [ ]:
webcam = cv2.VideoCapture(0)

while True:
    _, frame_bgr = webcam.read()
    orig_frame = frame_bgr.copy()
    cv2.imshow("Webcam", orig_frame)
    cv2.waitKey(1)
    time.sleep(0.02)  

In [ ]:
# When everything done, release the capture
webcam.release()
cv2.destroyAllWindows()

## Calibration Stuff
**reference size: 13x13cm**
* 34cm: 170
* 51cm: 120px
* 68cm: 93px
* 73cm: 77px

In [ ]:
import matplotlib.pyplot as plt
d = np.array([34,51,57,68,73,92])
h = np.array([170,120,110,93,78,62])
y = 5960/d
# for i in range(-10,11):
#     c = (6000 + i*10)
#     y = c/d
#     error = abs(np.sum(h**2 - y**2))**0.5
#     print(c,error)

plt.xlim(0,100)
plt.ylim(0,200)
plt.plot(d,h)
plt.plot(d,y)